# Init

In [1]:
from skfem import *
import numpy as np
from utils import *
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import datetime
import pandas as pd
import sys
import time


tol = 1e-8
intorder = 6
solver_type = 'mgcg'
refine_time = 7
element_type = 'P1'
sigma = 5
penalty = False
epsilon = 1e-6
ep = epsilon
example = 'ex3'
save_path = 'log/' + example + '_' + element_type + '_' + ('pen' if penalty else 'nopen') + '_' +'{}'.format(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

In [4]:
%%time
m = MeshTri()
m.refine(7)

if penalty:
    uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)
else:
    uh0, basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)

U = basis['u'].interpolate(uh0).value

# compute errors

L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
Du = get_DuError(basis['u'], uh0)
H1u = Du + L2u

if penalty:
    D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
else:
    D2u = get_D2uError(basis['u'], uh0)

epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)

Wall time: 3.54 s


In [5]:
uh0_store = uh0
basis_store = basis

In [6]:
%%time
m = MeshTri()
m.refine(3)

if penalty:
    uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)
else:
    uh0, basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)

U = basis['u'].interpolate(uh0).value

# compute errors

L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
Du = get_DuError(basis['u'], uh0)
H1u = Du + L2u

if penalty:
    D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
else:
    D2u = get_D2uError(basis['u'], uh0)

epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)

Wall time: 35.9 ms


In [7]:
L2u

0.024206825777796336

In [10]:
def exact_u_n(x, y):
    return basis_store['u'].interpolator(uh0_store)(np.array([[x], [y]]))

In [41]:
@Functional
def L2uError_h(w):
    x, y = w.x
    return (w.w - un)**2

In [16]:
np.sqrt(L2uError_h.assemble(basis['u'], w=U))

0.024206825777796336

In [42]:
np.sqrt(L2uError_h.assemble(basis['u'], w=U))

0.024196586358076023

In [39]:
x, y = www.x
exact_u(x, y)

array([[0.01703709, 0.01831591, 0.01831591, 0.00615583],
       [0.03378266, 0.02424519, 0.07213858, 0.01227371],
       [0.03378266, 0.07213858, 0.02424519, 0.01227371],
       [0.03378266, 0.02424519, 0.07213858, 0.01227371],
       [0.03378266, 0.07213858, 0.02424519, 0.01227371],
       [0.01703709, 0.01831591, 0.01831591, 0.00615583],
       [0.25660481, 0.3080644 , 0.30047883, 0.15595223],
       [0.98296291, 0.96937243, 0.96937243, 0.99384417],
       [0.12607862, 0.22201973, 0.07461903, 0.07749313],
       [0.9576622 , 0.94812473, 0.92477875, 0.98464363],
       [0.12940952, 0.23272573, 0.07629126, 0.07821723],
       [0.9576622 , 0.94812473, 0.92477875, 0.98464363],
       [0.12940952, 0.23272573, 0.07629126, 0.07821723],
       [0.12607862, 0.22201973, 0.07461903, 0.07749313],
       [0.25660481, 0.3080644 , 0.30047883, 0.15595223],
       [0.25660481, 0.30047883, 0.3080644 , 0.15595223],
       [0.12607862, 0.22201973, 0.07461903, 0.07749313],
       [0.12940952, 0.23272573,

In [36]:
from tqdm import tqdm

In [37]:
un = np.zeros_like(x)
for i in tqdm(range(x.shape[0])):
    for j in range(x.shape[1]):
        un[i][j] = basis_store['u'].interpolator(uh0_store)(np.array([[x[i][j]], [y[i][j]]]))

100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [03:14<00:00,  1.52s/it]


In [45]:
uh0.shape

(289,)

In [38]:
un

array([[0.01703435, 0.01831327, 0.01831327, 0.00615468],
       [0.03378406, 0.02424624, 0.07213967, 0.01227478],
       [0.03378406, 0.07213967, 0.02424624, 0.01227478],
       [0.03378406, 0.02424624, 0.07213967, 0.01227478],
       [0.03378406, 0.07213967, 0.02424624, 0.01227478],
       [0.01703435, 0.01831327, 0.01831327, 0.00615468],
       [0.25659032, 0.30804762, 0.3004604 , 0.15594434],
       [0.98291307, 0.96932337, 0.96932337, 0.99379405],
       [0.12607122, 0.22200551, 0.07461467, 0.07748961],
       [0.9576149 , 0.94807776, 0.92473427, 0.9845943 ],
       [0.1294041 , 0.23271426, 0.07628918, 0.07821405],
       [0.9576149 , 0.94807776, 0.92473427, 0.9845943 ],
       [0.1294041 , 0.23271426, 0.07628918, 0.07821405],
       [0.12607122, 0.22200551, 0.07461467, 0.07748961],
       [0.25659032, 0.30804762, 0.3004604 , 0.15594434],
       [0.25659032, 0.3004604 , 0.30804762, 0.15594434],
       [0.12607122, 0.22200551, 0.07461467, 0.07748961],
       [0.1294041 , 0.23271426,

In [3]:
U.shape

(32768, 4)

In [4]:
uh0.shape

(66049,)

In [6]:
basis['u'].interpolator(uh0)(np.array([[0.5],[0.5]]))

array([0.9999498])

In [57]:
m = MeshTri()
# m.refine(1)

if element_type == 'P1':
    element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
elif element_type == 'P2':
    element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
else:
    raise Exception("The element not supported")

basis = {
    variable: InteriorBasis(m, e, intorder=3)
    for variable, e in element.items()
}

In [32]:
# basis['u'].dx # weights
refine2 = basis['u'].global_coordinates().value

In [58]:
basis['u'].interpolator(uh0)(np.array([[0.5],[0.5]]))

array([1.94279276e-16])

# Solving

In [2]:
# solving 

print('=======Arguments=======')
print('penalty:\t{}'.format(penalty))
print('element_type:\t{}'.format(element_type))
print('solver_type:\t{}'.format(solver_type))
print('tol:\t{}'.format(tol))
print('intorder:\t{}'.format(intorder))
print('refine_time:\t{}'.format(refine_time))
print('epsilon:\t{}'.format(epsilon))
print('sigma:\t{}'.format(sigma))
print('=======Results=======')

time_start = time.time()

ep = epsilon
df_list = []
L2_list = []
Du_list = []
D2u_list = []
h_list = []
epu_list = []
m = MeshTri()

for i in range(1, refine_time+1):

    m.refine()

    if penalty:
        uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=6, tol=1e-8, epsilon=1e-6)
    else:
        uh0, basis = solve_problem1(m, element_type, solver_type, intorder=6, tol=1e-8, epsilon=1e-6)

    U = basis['u'].interpolate(uh0).value

    # compute errors

    L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
    Du = get_DuError(basis['u'], uh0)
    H1u = Du + L2u
    
    if penalty:
        D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
    else:
        D2u = get_D2uError(basis['u'], uh0)
        
    epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
    h_list.append(m.param())
    Du_list.append(Du)
    L2_list.append(L2u)
    D2u_list.append(D2u)
    epu_list.append(epu)

hs = np.array(h_list)
L2s = np.array(L2_list)
Dus = np.array(Du_list)
D2us = np.array(D2u_list)
epus = np.array(epu_list)
H1s = L2s + Dus
H2s = H1s + D2us

# store data
data = np.array([L2s, H1s, H2s, epus])
df = pd.DataFrame(data.T, columns=['L2', 'H1', 'H2', 'Energy'])
df_list.append(df)

show_result(L2s, H1s, H2s, epus)

# df.to_csv(save_path+'.csv')
time_end = time.time()
print('Total Time Cost {:.2f} s'.format(time_end-time_start))

=======Arguments=======
penalty:	False
element_type:	P1
solver_type:	mgcg
tol:	1e-08
intorder:	6
refine_time:	7
epsilon:	1e-06
sigma:	5
=======Results=======
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  1.74  0.81  -0.20  0.71
2^-2  7.631e-02  1.065e+00  1.309e+01  9.885e-01
2^-3  1.57  0.65  -0.46  0.60
2^-3  2.576e-02  6.781e-01  1.807e+01  6.523e-01
2^-4  1.51  0.55  -0.49  0.52
2^-4  9.051e-03  4.638e-01  2.543e+01  4.547e-01
2^-5  1.51  0.52  -0.49  0.50
2^-5  3.189e-03  3.240e-01  3.580e+01  3.208e-01
2^-6  1.50  0.51  -0.50  0.50
2^-6  1.124e-03  2.279e-01  5.047e+01  2.267e-01
2^-7  1.50  0.50  -0.50  0.50
2^-7  3.968e-04  1.607e-01  7.124e+01  1.603e-01
Total Time Cost 7.26 s
